# Prediction Model

In [1]:
import pandas as pd
from tools.utils import *
from sklearn.model_selection import train_test_split

In [2]:
games_model = pd.read_csv('data/games_model.csv')

In [3]:
df_train, df_test = train_test_split(games_model, stratify = games_model['WL_HOME'], test_size=0.2, random_state=7)

In [4]:
target = 'WL_HOME'
X_train = df_train.drop(columns = target)
y_train = df_train[target]

X_test = df_test.drop(columns = target)
y_test= df_test[target]

In [5]:
%pip install xgboost

  Using cached xgboost-1.7.5-py3-none-manylinux2014_x86_64.whl (200.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import xgboost as xgb

In [7]:
classifier = xgb.XGBClassifier(use_label_encoder=False, random_state=7)

/home/jovyan/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [8]:
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=7, ...)

In [9]:
from sklearn.metrics import accuracy_score
y_pred = classifier.predict(X_test)

In [10]:
accuracy_score(y_test, y_pred)

0.5785785785785785

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.fixes import loguniform

In [12]:
hyp_params = {'learning_rate': loguniform(0.0001, 1),
              'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
              'subsample': [0.7, 0.8, 0.9, 1.0],
              'n_estimators': [50, 100, 150, 200]}

In [13]:
random_hyp = RandomizedSearchCV(estimator =classifier, param_distributions=hyp_params, n_iter=20, cv=7,scoring='accuracy',random_state=7)

In [14]:
random_hyp.fit(X_train, y_train)

/home/jovyan/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/jovyan/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/jovyan/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/jovyan/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home/jovyan/envs/project/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
/home

RandomizedSearchCV(cv=7,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=7, ...),
                   n_iter=20,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3dbd0be8e0>,
                                        'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                                        'n_estimators': [50, 100, 150, 200],
                                        'subsample': [0.7, 0.8, 0.9, 1.0]},
                   random_state=7, scoring='accuracy')

In [15]:
random_hyp.best_params_

{'learning_rate': 0.0047587056428284825,
 'max_depth': 2,
 'n_estimators': 200,
 'subsample': 0.9}

In [16]:
model_hyp = random_hyp.best_estimator_

In [17]:
y_pred_hyp = model_hyp.predict(X_test)

In [18]:
accuracy_score(y_test, y_pred_hyp)

0.5975975975975976

In [19]:
# getting confusion matrix
cm_df = get_confusion_matrix(y_test, y_pred_hyp)

cm_df

,TP,FP,FN,TN
0,210,235,167,387


In [20]:
#calculating true positive rate and false positive rate
tpr, fpr = calculate_tpr_fpr(cm_df, row = 0)

print ("True positive rate:", tpr, "\nFalse positive rate:", fpr)

True positive rate: 55.7 
False positive rate: 37.78
